In [33]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.text import *
from fastai import *
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook as tqdm

# Preparing Model

In [2]:
PATH = Path('data/').absolute()

In [6]:
bs = 48

In [7]:
data_clas = load_data(PATH, 'classifier_train_data', bs=bs)

In [8]:
# Reclaiming up some memory
learn = None
gc.collect()

learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('../Data_Processed/Data_Classifier/models/fine_tuned_enc')

In [9]:
def roc_score(model=learn):
    """
    Calculates the ROC score for all classes separately and returns their mean.
    """
    y_preds, y_true = learn.get_preds()
    return roc_auc_score(y_true.numpy(), y_preds.numpy(), average='macro')

In [10]:
# This metric will classify as correct any prediction above `thresh = 0.4`.
# We will then simply calculate the accuracy taking into account each label as in the normal accuracy measure
acc = partial(accuracy_thresh, thresh=0.4)
learn.metrics.append(acc)

In [11]:
learn.load('unfrozen_2')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (143614 items)
x: TextList
xxbos xxmaj explanation 
  xxmaj why the edits made under my username xxmaj hardcore xxmaj metallica xxmaj fan were reverted ? xxmaj they were n't vandalisms , just closure on some gas after i voted at xxmaj new xxmaj york xxmaj dolls xxup fac . xxmaj and please do n't remove the template from the talk page since i 'm retired xxunk,xxbos xxmaj xxunk ! xxmaj he matches this background colour i 'm seemingly stuck with . xxmaj thanks . ( talk ) 21:51 , xxmaj january 11 , 2016 ( xxup utc ),xxbos xxmaj hey man , i 'm really not trying to edit war . xxmaj it 's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page . xxmaj he seems to care more about the formatting than the actual info .,xxbos " 
  xxmaj more 
  i ca n't make any real suggestions on improvement - i wondered if the section statistics should be later on , or a subsection of " " types o

# Loading Test Data

In [14]:
df = pd.read_csv(PATH/'test.csv')
df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [46]:
learn.data.add_test(df.iloc[:,1])

In [55]:
preds = learn.get_preds(DatasetType.Test)

predictions = preds[0].numpy()

predictions = pd.DataFrame(predictions, 
                           columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [67]:
submission = df.join(predictions).drop('comment_text', axis=1)

In [68]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.988465,3.144439e-01,0.952475,0.088045,0.879941,0.257713
1,0000247867823ef7,0.000085,1.392482e-05,0.000092,0.000013,0.000053,0.000031
2,00013b17ad220c46,0.002955,1.442172e-04,0.001281,0.000281,0.000693,0.000299
3,00017563c3f7919a,0.000001,2.555014e-07,0.000002,0.000002,0.000003,0.000002
4,00017695ad8997eb,0.019467,9.590147e-05,0.002466,0.000423,0.001411,0.000398


In [70]:
submission.to_csv(PATH/'kaggle_submission_1.csv', index=False)

In [71]:
len(submission)

153164